# Preparación de entorno y dataset

In [ ]:
! pip install pyarrow
! pip install -q kaggle

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [ ]:
pd.options.display.float_format = '{:20,.4f}'.format # suprimimos la notacion cientifica en los outputs

In [ ]:
from google.colab import files

files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"hangyeolkoo","key":"4032f7e1f591049aec886c05d7504de2"}'}

In [ ]:
! mkdir ~/.kaggle

In [ ]:
! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle competitions download vpn-classification -f dataset_v2/train.parq

100% 564M/564M [00:21<00:00, 31.7MB/s]
100% 564M/564M [00:21<00:00, 28.0MB/s]


In [ ]:
! kaggle competitions download vpn-classification -f dataset_v2/test.parq

100% 164M/164M [00:06<00:00, 34.9MB/s]
100% 164M/164M [00:06<00:00, 28.1MB/s]


In [ ]:
! kaggle competitions download vpn-classification -f dataset_v2/shodan_df_hashed.csv

 47% 1.00M/2.11M [00:00<00:00, 2.00MB/s]
100% 2.11M/2.11M [00:00<00:00, 3.56MB/s]


In [ ]:
! unzip train.parq.zip

Archive:  train.parq.zip
  inflating: train.parq              


In [ ]:
! unzip test.parq.zip

Archive:  test.parq.zip
  inflating: test.parq               


In [ ]:
! unzip shodan_df_hashed.csv.zip

Archive:  shodan_df_hashed.csv.zip
  inflating: shodan_df_hashed.csv    


## Manejo de train dataset

In [ ]:
train = pd.read_parquet("train.parq",engine="auto")
test = pd.read_parquet("test.parq",engine="auto")

**Imputo los valores nulos**

Por cuestiones de perfomance se decidió crear un propio Imputer, que rellena los NaNs con los datos más frecuentes. Esto se llevó a cabo sólo en dataset de test, para no utilizar informaciones erróneas para el entrenamiento.

In [ ]:
class MostFrequentImputer:
    def __init__(self):
        self.most_frequent_values = None

    def fit(self, df):
        self.most_frequent_values = {col: df[col].mode().iloc[0] for col in df.columns}
        return self

    def transform(self, df):
        for col, value in self.most_frequent_values.items():
            df[col] = df[col].fillna(value)
        return df

In [ ]:
imputer = MostFrequentImputer()

test_columns = test.columns

imputer.fit(test)
imputer.transform(test)

test.isna().sum().sum()

0

In [ ]:
train.head()

,attack_time,watcher_country,watcher_as_num,watcher_as_name,attacker_country,attacker_as_num,attacker_as_name,attack_type,watcher_uuid_enum,attacker_ip_enum,label
0,2023-07-31 07:17:51+00:00,DE,"34,011.0000",Host Europe GmbH,TR,"47,721.0000",Murat Aktas,http:exploit,0,6466,0
1,2023-07-31 07:17:51+00:00,DE,"34,011.0000",Host Europe GmbH,TR,"47,721.0000",Murat Aktas,http:spam,0,6466,0
2,2023-07-31 07:17:49+00:00,DE,"20,886.0000",bn:t Blatzheim Networks Telecom GmbH,DE,"51,167.0000",Contabo GmbH,http:bruteforce,2,4637,0
3,2023-07-31 07:17:49+00:00,DE,"20,886.0000",bn:t Blatzheim Networks Telecom GmbH,DE,"51,167.0000",Contabo GmbH,http:spam,2,4637,0
4,2023-07-31 07:17:49+00:00,DE,"20,886.0000",bn:t Blatzheim Networks Telecom GmbH,DE,"51,167.0000",Contabo GmbH,http:exploit,2,4637,0


In [ ]:
train.isna().sum().sum()

822426

In [ ]:
train['label'].value_counts()

0    60594448
1     1035237
Name: label, dtype: int64

In [ ]:
train = train.dropna()

In [ ]:
train['label'].value_counts()

0    60135084
1     1026672
Name: label, dtype: int64

In [ ]:
train = train.drop(columns=['watcher_as_name','attacker_as_name'])

test = test.drop(columns=['watcher_as_name','attacker_as_name'])

train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 61161756 entries, 0 to 61629684
Data columns (total 9 columns):
 #   Column             Dtype              
---  ------             -----              
 0   attack_time        datetime64[ns, UTC]
 1   watcher_country    category           
 2   watcher_as_num     float32            
 3   attacker_country   category           
 4   attacker_as_num    float32            
 5   attack_type        category           
 6   watcher_uuid_enum  int32              
 7   attacker_ip_enum   int32              
 8   label              int8               
dtypes: category(3), datetime64[ns, UTC](1), float32(2), int32(2), int8(1)
memory usage: 2.2 GB


Las columnas que aportan informaciones equivalentes a otra ya existente fueron descartadas.

## Feature engineering

In [ ]:
train['day_of_week'] = train['attack_time'].dt.day_name()
train['hour'] = train['attack_time'].dt.hour

test['day_of_week'] = test['attack_time'].dt.day_name()
test['hour'] = test['attack_time'].dt.hour

In [ ]:
train[['service', 'threat_type']] = train['attack_type'].str.split(':', expand=True)

test[['service', 'threat_type']] = test['attack_type'].str.split(':', expand=True)

In [ ]:
train['same_country'] = (train['attacker_country'].astype('str') == train['watcher_country'].astype('str')).astype(int)

test['same_country'] = (test['attacker_country'].astype('str') == test['watcher_country'].astype('str')).astype(int)

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 61161756 entries, 0 to 61629684
Data columns (total 14 columns):
 #   Column             Dtype              
---  ------             -----              
 0   attack_time        datetime64[ns, UTC]
 1   watcher_country    category           
 2   watcher_as_num     float32            
 3   attacker_country   category           
 4   attacker_as_num    float32            
 5   attack_type        category           
 6   watcher_uuid_enum  int32              
 7   attacker_ip_enum   int32              
 8   label              int8               
 9   day_of_week        object             
 10  hour               int64              
 11  service            object             
 12  threat_type        object             
 13  same_country       int64              
dtypes: category(3), datetime64[ns, UTC](1), float32(2), int32(2), int64(2), int8(1), object(3)
memory usage: 4.4+ GB


In [ ]:
train['hour'] = train['hour'].astype('int8')
train['service'] = train['service'].astype('category')
train['threat_type'] = train['threat_type'].astype('category')
train['same_country'] = train['same_country'].astype('category')

test['hour'] = test['hour'].astype('int8')
test['service'] = test['service'].astype('category')
test['threat_type'] = test['threat_type'].astype('category')
test['same_country'] = test['same_country'].astype('category')

In [ ]:
def map_day_to_sin(day):
  list_days = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']
  for i in range(7):
    if day == list_days[i]:
      return np.sin(2*np.pi*i/7)
  return day

In [ ]:
def map_hour_to_sin(hour):
  return np.sin(2*np.pi*hour/7)

In [ ]:
train['sin_day_of_week'] = train['day_of_week'].map(map_day_to_sin)
train['sin_hour'] = train['hour'].map(map_hour_to_sin)

test['sin_day_of_week'] = test['day_of_week'].map(map_day_to_sin)
test['sin_hour'] = test['hour'].map(map_hour_to_sin)

In [ ]:
train = train.drop(columns=['day_of_week','hour'])

test = test.drop(columns=['day_of_week','hour'])

In [ ]:
train['sin_day_of_week'] = train['sin_day_of_week'].astype('float')

test['sin_day_of_week'] = test['sin_day_of_week'].astype('float')

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 61161756 entries, 0 to 61629684
Data columns (total 14 columns):
 #   Column             Dtype              
---  ------             -----              
 0   attack_time        datetime64[ns, UTC]
 1   watcher_country    category           
 2   watcher_as_num     float32            
 3   attacker_country   category           
 4   attacker_as_num    float32            
 5   attack_type        category           
 6   watcher_uuid_enum  int32              
 7   attacker_ip_enum   int32              
 8   label              int8               
 9   service            category           
 10  threat_type        category           
 11  same_country       category           
 12  sin_day_of_week    float64            
 13  sin_hour           float64            
dtypes: category(6), datetime64[ns, UTC](1), float32(2), float64(2), int32(2), int8(1)
memory usage: 3.2 GB


In [ ]:
train = train.drop(columns=['attack_time'])

test = test.drop(columns=['attack_time'])

In [ ]:
train.head()

,watcher_country,watcher_as_num,attacker_country,attacker_as_num,attack_type,watcher_uuid_enum,attacker_ip_enum,label,service,threat_type,same_country,sin_day_of_week,sin_hour
0,DE,"34,011.0000",TR,"47,721.0000",http:exploit,0,6466,0,http,exploit,0,0.0000,-0.0000
1,DE,"34,011.0000",TR,"47,721.0000",http:spam,0,6466,0,http,spam,0,0.0000,-0.0000
2,DE,"20,886.0000",DE,"51,167.0000",http:bruteforce,2,4637,0,http,bruteforce,1,0.0000,-0.0000
3,DE,"20,886.0000",DE,"51,167.0000",http:spam,2,4637,0,http,spam,1,0.0000,-0.0000
4,DE,"20,886.0000",DE,"51,167.0000",http:exploit,2,4637,0,http,exploit,1,0.0000,-0.0000


In [ ]:
train['attack_count_per_ip'] = train.groupby('attacker_ip_enum')['attacker_ip_enum'].transform('count')

test['attack_count_per_ip'] = test.groupby('attacker_ip_enum')['attacker_ip_enum'].transform('count')

In [ ]:
train_grouped_by_ip_enum = train.groupby('attacker_ip_enum').agg({
    'attack_count_per_ip' : 'first',
    'watcher_country': lambda x: x.mode().iat[0],
    'watcher_as_num': lambda x: x.mode().iat[0],
    'attacker_country': lambda x: x.mode().iat[0],
    'attacker_as_num': lambda x: x.mode().iat[0],
    'attack_type': lambda x: x.mode().iat[0],
    'watcher_uuid_enum': lambda x: x.mode().iat[0],
    'label': lambda x: x.mode().iat[0],
    'sin_day_of_week': 'mean',
    'sin_hour': 'mean',
    'service': lambda x: x.mode().iat[0],
    'threat_type': lambda x: x.mode().iat[0],
    'same_country': lambda x: x.mode().iat[0]
}).reset_index()

test_grouped_by_ip_enum = test.groupby('attacker_ip_enum').agg({
    'attack_count_per_ip' : 'first',
    'watcher_country': lambda x: x.mode().iat[0],
    'watcher_as_num': lambda x: x.mode().iat[0],
    'attacker_country': lambda x: x.mode().iat[0],
    'attacker_as_num': lambda x: x.mode().iat[0],
    'attack_type': lambda x: x.mode().iat[0],
    'watcher_uuid_enum': lambda x: x.mode().iat[0],
    'sin_day_of_week': 'mean',
    'sin_hour': 'mean',
    'service': lambda x: x.mode().iat[0],
    'threat_type': lambda x: x.mode().iat[0],
    'same_country': lambda x: x.mode().iat[0]
}).reset_index()

train_grouped_by_ip_enum.head()

,attacker_ip_enum,attack_count_per_ip,watcher_country,watcher_as_num,attacker_country,attacker_as_num,attack_type,watcher_uuid_enum,label,sin_day_of_week,sin_hour,service,threat_type,same_country
0,0,52,US,"14,061.0000",US,"14,618.0000",http:scan,18401,0,-0.4339,0.0816,http,scan,0
1,1,30,US,"3,303.0000",BR,"27,699.0000",http:exploit,8646,0,0.7818,0.1567,http,exploit,0
2,2,58,DE,"12,897.0000",DE,"3,320.0000",http:scan,24879,0,0.0890,0.0876,http,scan,1
3,3,22,DE,"12,897.0000",VE,"21,826.0000",http:exploit,24879,0,-0.1423,0.2312,http,exploit,0
4,4,68,US,"396,982.0000",SA,"25,019.0000",http:exploit,1828,0,-0.2439,0.2687,http,exploit,0


In [ ]:
train_grouped_by_ip_enum.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 147649 entries, 0 to 147648
Data columns (total 14 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   attacker_ip_enum     147649 non-null  int64  
 1   attack_count_per_ip  147649 non-null  int64  
 2   watcher_country      147649 non-null  object 
 3   watcher_as_num       147649 non-null  float32
 4   attacker_country     147649 non-null  object 
 5   attacker_as_num      147649 non-null  float32
 6   attack_type          147649 non-null  object 
 7   watcher_uuid_enum    147649 non-null  int32  
 8   label                147649 non-null  int8   
 9   sin_day_of_week      147649 non-null  float64
 10  sin_hour             147649 non-null  float64
 11  service              147649 non-null  object 
 12  threat_type          147649 non-null  object 
 13  same_country         147649 non-null  int64  
dtypes: float32(2), float64(2), int32(1), int64(3), int8(1), object(5)
me

Dado que LightGBM no puede recibir columnas con dtype object, los encodeamos como corresponde

In [ ]:
train_grouped_by_ip_enum['watcher_country'] = train_grouped_by_ip_enum['watcher_country'].astype('category')
train_grouped_by_ip_enum['watcher_as_num'] = train_grouped_by_ip_enum['watcher_as_num'].astype('category')
train_grouped_by_ip_enum['attacker_country'] = train_grouped_by_ip_enum['attacker_country'].astype('category')
train_grouped_by_ip_enum['attacker_as_num'] = train_grouped_by_ip_enum['attacker_as_num'].astype('category')
train_grouped_by_ip_enum['watcher_uuid_enum'] = train_grouped_by_ip_enum['watcher_uuid_enum'].astype('category')
train_grouped_by_ip_enum['attack_type'] = train_grouped_by_ip_enum['attack_type'].astype('category')
train_grouped_by_ip_enum['service'] = train_grouped_by_ip_enum['service'].astype('category')
train_grouped_by_ip_enum['threat_type'] = train_grouped_by_ip_enum['threat_type'].astype('category')
train_grouped_by_ip_enum['same_country'] = train_grouped_by_ip_enum['same_country'].astype('int8')

test_grouped_by_ip_enum['watcher_country'] = test_grouped_by_ip_enum['watcher_country'].astype('category')
test_grouped_by_ip_enum['watcher_as_num'] = test_grouped_by_ip_enum['watcher_as_num'].astype('category')
test_grouped_by_ip_enum['attacker_country'] = test_grouped_by_ip_enum['attacker_country'].astype('category')
test_grouped_by_ip_enum['attacker_as_num'] = test_grouped_by_ip_enum['attacker_as_num'].astype('category')
test_grouped_by_ip_enum['watcher_uuid_enum'] = test_grouped_by_ip_enum['watcher_uuid_enum'].astype('category')
test_grouped_by_ip_enum['attack_type'] = test_grouped_by_ip_enum['attack_type'].astype('category')
test_grouped_by_ip_enum['service'] = test_grouped_by_ip_enum['service'].astype('category')
test_grouped_by_ip_enum['threat_type'] = test_grouped_by_ip_enum['threat_type'].astype('category')
test_grouped_by_ip_enum['same_country'] = test_grouped_by_ip_enum['same_country'].astype('int8')

## Manejo de shodan_info



In [ ]:
shodan_df = pd.read_csv('shodan_df_hashed.csv')

In [ ]:
shodan_df.head()

,shodan_info,attacker_ip_enum
0,{},5915
1,"{'22/tcp': {'headers_hash': None, 'jarm': None...",3325
2,{},8416
3,{},1213
4,{},9185


In [ ]:
len(shodan_df) == len(shodan_df['attacker_ip_enum'].unique())

True

In [ ]:
import ast
from collections import Counter

port_counter = Counter()
ip_port_list = {}

for index, row in shodan_df.iterrows():
  shodan_info = ast.literal_eval(row['shodan_info'])
  ip_port_list[int(row['attacker_ip_enum'])] = list(shodan_info)
  port_counter.update(shodan_info.keys())

In [ ]:
ip_port_count = {}
for ip in ip_port_list:
  ip_port_count[ip] = len(ip_port_list[ip])

In [ ]:
len(port_counter.keys())

2297

In [ ]:
port_counter.most_common()[10]

('587/tcp', 2300)

In [ ]:
top_port = set()
for port in port_counter.most_common(10):
  top_port.add(port[0])

In [ ]:
for ip in ip_port_list:
  for port in ip_port_list[ip]:
    if port not in top_port:
      ip_port_list[ip].remove(port)

In [ ]:
train_grouped_by_ip_enum['port_count'] = train_grouped_by_ip_enum['attacker_ip_enum'].map(ip_port_count)
train_grouped_by_ip_enum['port_list'] = train_grouped_by_ip_enum['attacker_ip_enum'].map(ip_port_list)

test_grouped_by_ip_enum['port_count'] = test_grouped_by_ip_enum['attacker_ip_enum'].map(ip_port_count)
test_grouped_by_ip_enum['port_list'] = test_grouped_by_ip_enum['attacker_ip_enum'].map(ip_port_list)

In [ ]:
for port in top_port:
  train_grouped_by_ip_enum[port] = train_grouped_by_ip_enum['port_list'].map(lambda port_list: int(port in port_list))
  test_grouped_by_ip_enum[port] = test_grouped_by_ip_enum['port_list'].map(lambda port_list: int(port in port_list))

In [ ]:
train_grouped_by_ip_enum = train_grouped_by_ip_enum.drop(columns='port_list')
test_grouped_by_ip_enum = test_grouped_by_ip_enum.drop(columns='port_list')

## Dataset resultante

En resumen, se crearon las siguientes nuevas features:
* Cantidad de ataques por attacker_ip_enum
* Seno de día de semana(siendo 0 lunes, 6 domingo)
* Seno de hora de ataque
* Servicio atacado
* Tipo de amenaza (spam, scam, etc.)
* Si el país atacante y atacado coinciden
* Cantidad de puertos

Este último fue creado teniendo en cuenta que en caso de coincidencia, será más probable que el atacante no esté utilizando un VPN, que generalmente cambia la ubicación.

In [ ]:
train_grouped_by_ip_enum.head()

,attacker_ip_enum,attack_count_per_ip,watcher_country,watcher_as_num,attacker_country,attacker_as_num,attack_type,watcher_uuid_enum,label,sin_day_of_week,...,22/tcp,995/tcp,2000/tcp,993/tcp,53/udp,53/tcp,21/tcp,465/tcp,80/tcp,443/tcp
0,0,52,US,"14,061.0000",US,"14,618.0000",http:scan,18401,0,-0.4339,...,0,0,0,0,0,0,0,0,0,0
1,1,30,US,"3,303.0000",BR,"27,699.0000",http:exploit,8646,0,0.7818,...,0,0,0,0,0,0,0,0,0,0
2,2,58,DE,"12,897.0000",DE,"3,320.0000",http:scan,24879,0,0.0890,...,0,0,0,0,0,0,0,0,0,0
3,3,22,DE,"12,897.0000",VE,"21,826.0000",http:exploit,24879,0,-0.1423,...,0,0,0,0,0,0,0,0,0,0
4,4,68,US,"396,982.0000",SA,"25,019.0000",http:exploit,1828,0,-0.2439,...,0,0,0,0,0,0,0,0,0,0


## Split

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
y = train_grouped_by_ip_enum['label']
X = train_grouped_by_ip_enum.drop(columns=['label'])

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=12)

In [ ]:
def common_member(a, b):
    a_set = set(a)
    b_set = set(b)

    if (a_set & b_set):
        print(a_set & b_set)
    else:
        print("No common elements")

common_member(X_train['attacker_ip_enum'],X_valid['attacker_ip_enum'])

No common elements


In [ ]:
X_train = X_train.set_index('attacker_ip_enum')
X_valid = X_valid.set_index('attacker_ip_enum')
X_test = test_grouped_by_ip_enum.set_index('attacker_ip_enum')

# Modelo


## Modelo con parámetros default

In [ ]:
import lightgbm as lgb

In [ ]:
categorical_features = ['watcher_country','watcher_as_num','attacker_country','attacker_as_num','attack_type','watcher_uuid_enum','service','threat_type']

In [ ]:
model_default = lgb.LGBMClassifier(
    objective='binary',
    metric='f1',
    seed=12,
    categorical_feature=categorical_features
)

In [ ]:
model_default.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/lightgbm/basic.py:1873: UserWarning: categorical_feature keyword has been found in `params` and will be ignored.
Please use categorical_feature argument of the Dataset constructor to pass this parameter.
  _log_warning(f'{key} keyword has been found in `params` and will be ignored.\n'
/usr/local/lib/python3.10/dist-packages/lightgbm/basic.py:1893: UserWarning: categorical_feature in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Warning] categorical_feature is set=watcher_country,watcher_as_num,attacker_country,attacker_as_num,attack_type,watcher_uuid_enum,service,threat_type, categorical_column=1,2,3,4,5,6,9,10 will be ignored. Current value: categorical_feature=watcher_country,watcher_as_num,attacker_country,attacker_as_num,attack_type,watcher_uuid_enum,service,threat_type
[LightGBM] [Info] Number of positive: 2285, number of negative: 115834
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.009800 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 11081
[LightGBM] [Info] Number of data points in the train set: 118119, number

LGBMClassifier(categorical_feature=['watcher_country', 'watcher_as_num',
                                    'attacker_country', 'attacker_as_num',
                                    'attack_type', 'watcher_uuid_enum',
                                    'service', 'threat_type'],
               metric='f1', objective='binary', seed=12)

In [ ]:
from sklearn.metrics import f1_score

y_pred = model_default.predict(X_valid)

f1_score(y_valid, y_pred)

0.759927797833935

In [ ]:
y_pred_train_default = model_default.predict(X_train)

f1_score(y_train, y_pred_train_default)

0.9469980206729711

## Modelo con parámetros encontrados mediante Random Search

In [ ]:
param_dist = {
    'n_estimators':[3000, 5000, 8000],
    'learning_rate':[0.006, 0.005, 0.004],
    'max_depth':[7, 8],
    'num_leaves':[80,150],
    'bagging_fraction':[0.75, 0.8, 0.85],
    'metric':['f1'],
    'objective':['binary'],
    'is_unbalance':[True]
}

In [ ]:
lgb_classifier = lgb.LGBMClassifier()

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

random_search = RandomizedSearchCV(
    estimator=lgb_classifier,
    param_distributions=param_dist,
    scoring='f1',
    n_iter=50,
    cv=2,
    random_state=12,
    n_jobs=-1
)

In [ ]:
random_search.fit(X_train, y_train)

In [ ]:
best_model = random_search.best_estimator_
best_model

In [ ]:
y_pred = best_model.predict(X_valid)

f1_score(y_valid, y_pred)

In [ ]:
y_pred_train = best_model.predict(X_train)

f1_score(y_train, y_pred_train)

## Modelo con parámetros ingresados manualmente

In [ ]:
model_manual = lgb.LGBMClassifier(bagging_fraction=0.8,
                                  bagging_freq=10,
                                  learning_rate=0.005,
                                  max_depth=7,
                                  num_leaves=80,
                                  n_estimators=10000,
                                  objective='binary',
                                  metric='f1',
                                  is_unbalance=True,
                                  random_state=12)

In [ ]:
model_manual2 = lgb.LGBMClassifier(bagging_fraction=0.8,
                                  learning_rate=0.006,
                                  max_depth=8,
                                  num_leaves=150,
                                  n_estimators=5000,
                                  objective='binary',
                                  metric='f1',
                                  is_unbalance=True,
                                  random_state=12)

In [ ]:
model_manual.fit(X_train, y_train)

In [ ]:
model_manual2.fit(X_train, y_train)

[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Info] Number of positive: 2285, number of negative: 115834
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.008151 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 11081
[LightGBM] [Info] Number of data points in the train set: 118119, number of used features: 23
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.019345 -> initscore=-3.925792
[LightGBM] [Info] Start train

LGBMClassifier(bagging_fraction=0.8, is_unbalance=True, learning_rate=0.006,
               max_depth=8, metric='f1', n_estimators=5000, num_leaves=150,
               objective='binary', random_state=12)

In [ ]:
y_pred_manual = model_manual.predict(X_valid)

f1_score(y_valid, y_pred_manual)

In [ ]:
y_pred_train_manual = model_manual.predict(X_train)

f1_score(y_train, y_pred_train_manual)

In [ ]:
y_pred_manual2 = model_manual2.predict(X_valid)

f1_score(y_valid, y_pred_manual2)

[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8


0.7833333333333334

In [ ]:
y_pred_train_manual2 = model_manual2.predict(X_train)

f1_score(y_train, y_pred_train_manual2)

[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8


0.9978165938864628

## Test dataset

In [ ]:
from google.colab import drive

drive.mount('drive')

Mounted at drive


In [ ]:
prediction_manual = pd.Series(model_manual2.predict(X_test), index=X_test.index).rename("prediction")

[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8


In [ ]:
prediction_manual.to_csv("predictions_lightgbm_manual4.csv")

! cp predictions_lightgbm_manual4.csv "drive/My Drive/*Organización de Datos/TP2/predictions/"

# Preguntas

¿Cómo conviene elegir los datos de validación respecto de los de train?
> Es conveniente elegir ips distintos para cada dataset ya que dado que un mismo ip es siempre o vpn o no, puede generar overfitting donde el modelo simplemente 'memoriza'.

¿Cuál es el mejor score en la competencia?
> El mejor score obtenido es 0.74233.

# Archivos CSV de las predicciones

Los archivos csv de las predicciones se encuentran en el siguiente enlace: https://drive.google.com/drive/folders/189OwWzV26vH0L5Ubf3POpyDzAz8BbloI?usp=sharing